In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio.Data import CodonTable
from mpl_toolkits.mplot3d import Axes3D

In [2]:
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
print(standard_table)

Table 1 Standard, SGC0

  |  T      |  C      |  A      |  G      |
--+---------+---------+---------+---------+--
T | TTT F   | TCT S   | TAT Y   | TGT C   | T
T | TTC F   | TCC S   | TAC Y   | TGC C   | C
T | TTA L   | TCA S   | TAA Stop| TGA Stop| A
T | TTG L(s)| TCG S   | TAG Stop| TGG W   | G
--+---------+---------+---------+---------+--
C | CTT L   | CCT P   | CAT H   | CGT R   | T
C | CTC L   | CCC P   | CAC H   | CGC R   | C
C | CTA L   | CCA P   | CAA Q   | CGA R   | A
C | CTG L(s)| CCG P   | CAG Q   | CGG R   | G
--+---------+---------+---------+---------+--
A | ATT I   | ACT T   | AAT N   | AGT S   | T
A | ATC I   | ACC T   | AAC N   | AGC S   | C
A | ATA I   | ACA T   | AAA K   | AGA R   | A
A | ATG M(s)| ACG T   | AAG K   | AGG R   | G
--+---------+---------+---------+---------+--
G | GTT V   | GCT A   | GAT D   | GGT G   | T
G | GTC V   | GCC A   | GAC D   | GGC G   | C
G | GTA V   | GCA A   | GAA E   | GGA G   | A
G | GTG V   | GCG A   | GAG E   | GGG G   | G
--+---------

In [3]:
codon_table = standard_table.forward_table
for codon in standard_table.stop_codons:
    codon_table[codon] = 'X'
amino_acid = np.array(list(codon_table.values()))
codons = np.array(list(codon_table.keys()))

#for epoch in range(100):
#    f = open('random_genetic_code/code_' + str(epoch) + '.txt', 'w')
#    np.random.shuffle(amino_acid)
#    np.random.shuffle(codons)
#    for i in range(len(codons)):
#        f.write("%s\t%s\n" % (codons[i], amino_acid[i]))
#    f.close()

In [4]:
np.random.shuffle(amino_acid)
np.random.shuffle(codons)

In [5]:
codons

array(['GTG', 'GAC', 'TGG', 'AAG', 'ACC', 'CAT', 'ATG', 'GTC', 'ATC',
       'TTA', 'TCA', 'CTG', 'CAG', 'GCC', 'GAT', 'GGT', 'AGC', 'TCG',
       'AGG', 'ATT', 'CGC', 'CCT', 'TGT', 'GCT', 'TGA', 'AAT', 'GGA',
       'TTG', 'ACA', 'CTC', 'AAA', 'CTA', 'CGT', 'GCA', 'CTT', 'GTT',
       'TGC', 'CGG', 'TAC', 'ACT', 'TAT', 'TTC', 'AGA', 'CAA', 'ACG',
       'GGC', 'AAC', 'GAG', 'AGT', 'ATA', 'GTA', 'TAG', 'GCG', 'GGG',
       'TTT', 'CCG', 'CCA', 'TCC', 'TAA', 'CAC', 'CCC', 'GAA', 'TCT', 'CGA'],
      dtype='<U3')

In [6]:
def distance(A1, A2):
    D = 0
    for k in range(3):
        # transition
        if set([A1[k], A2[k]]) in [set(['A', 'G']), set(['C', 'T'])]:
            if k == 0 and A1[k] != A2[k]:
                D += 5
            elif k == 1 and A1[k] != A2[k]:
                D += 10
            elif k == 2 and A1[k] != A2[k]:
                D += 1
        # transversion
        else:
            if k == 0 and A1[k] != A2[k]:
                D += 10
            elif k == 1 and A1[k] != A2[k]:
                D += 20
            elif k == 2 and A1[k] != A2[k]:
                D += 2
    return(D)

In [7]:
cd = sorted(list(codon_table.keys()))
aa = list(set(codon_table.values()))
cd_num = len(cd)
aa_num = len(aa)

In [8]:
# create a distance matrix
D = np.zeros([cd_num, cd_num])
for i in range(cd_num):
    for j in range(cd_num):
        D[i, j] = distance(cd[i], cd[j])
D = D.astype(int)
pd.DataFrame(D, index=cd, columns=cd)

,AAA,AAC,AAG,AAT,ACA,ACC,ACG,ACT,AGA,AGC,...,TCG,TCT,TGA,TGC,TGG,TGT,TTA,TTC,TTG,TTT
AAA,0,2,1,2,20,22,21,22,10,12,...,31,32,20,22,21,22,30,32,31,32
AAC,2,0,2,1,22,20,22,21,12,10,...,32,31,22,20,22,21,32,30,32,31
AAG,1,2,0,2,21,22,20,22,11,12,...,30,32,21,22,20,22,31,32,30,32
AAT,2,1,2,0,22,21,22,20,12,11,...,32,30,22,21,22,20,32,31,32,30
ACA,20,22,21,22,0,2,1,2,20,22,...,11,12,30,32,31,32,20,22,21,22
ACC,22,20,22,21,2,0,2,1,22,20,...,12,11,32,30,32,31,22,20,22,21
ACG,21,22,20,22,1,2,0,2,21,22,...,10,12,31,32,30,32,21,22,20,22
ACT,22,21,22,20,2,1,2,0,22,21,...,12,10,32,31,32,30,22,21,22,20
AGA,10,12,11,12,20,22,21,22,0,2,...,31,32,10,12,11,12,30,32,31,32
AGC,12,10,12,11,22,20,22,21,2,0,...,32,31,12,10,12,11,32,30,32,31


In [33]:
K = np.random.random([len(cd), len(aa)])
V = np.copy(K)
pd.DataFrame(K, index=cd, columns=aa)

,Y,P,D,G,N,R,E,Q,M,K,...,S,A,W,L,I,C,X,V,F,H
AAA,0.112846,0.001927,0.833033,0.326726,0.364662,0.738994,0.274795,0.473870,0.318928,0.067806,...,0.561949,0.885107,0.962724,0.802923,0.931641,0.855531,0.166800,0.447096,0.526500,0.491966
AAC,0.266071,0.012524,0.013487,0.456161,0.789841,0.146614,0.127767,0.604428,0.022789,0.484328,...,0.691136,0.410675,0.604349,0.664625,0.164868,0.326961,0.892961,0.461281,0.279751,0.396698
AAG,0.178485,0.545314,0.833766,0.013147,0.260511,0.322018,0.615141,0.487755,0.877301,0.116517,...,0.171340,0.501590,0.063980,0.281306,0.287685,0.508761,0.641090,0.002251,0.907936,0.433686
AAT,0.452804,0.513652,0.614403,0.257467,0.152853,0.018069,0.436224,0.865325,0.736186,0.798698,...,0.173243,0.468367,0.416107,0.684088,0.678469,0.603353,0.507617,0.627864,0.563598,0.801970
ACA,0.796974,0.660032,0.291137,0.960673,0.938626,0.928730,0.904430,0.415800,0.766708,0.447386,...,0.992549,0.420045,0.452292,0.984365,0.464303,0.140672,0.396745,0.768359,0.510947,0.385431
ACC,0.042605,0.275921,0.311618,0.122898,0.000238,0.537030,0.030821,0.180944,0.185718,0.520114,...,0.727269,0.344237,0.889326,0.793286,0.392831,0.544694,0.248041,0.408359,0.194432,0.309543
ACG,0.149447,0.496287,0.713672,0.297364,0.204500,0.445384,0.465083,0.879259,0.269278,0.614506,...,0.073625,0.727872,0.221266,0.924372,0.504974,0.942090,0.877998,0.293615,0.080238,0.918427
ACT,0.974481,0.109624,0.542498,0.422765,0.015671,0.785436,0.646825,0.112551,0.403549,0.104530,...,0.835132,0.774214,0.964978,0.746902,0.927678,0.546054,0.451925,0.835385,0.274196,0.119686
AGA,0.125117,0.087869,0.795935,0.305015,0.001836,0.395335,0.024451,0.798583,0.064467,0.373984,...,0.134015,0.740305,0.888720,0.530998,0.282119,0.211198,0.782346,0.548760,0.355364,0.661180
AGC,0.180282,0.830387,0.820643,0.171269,0.477634,0.478201,0.213560,0.725656,0.551469,0.470563,...,0.822437,0.684905,0.993130,0.582009,0.879546,0.442411,0.932888,0.157484,0.447401,0.722921


In [10]:
def I(i):
    if i + 1 > aa_num - 1:
        return(i - 1, 0)
    elif i - 1 < 0:
        return(aa_num - 1, i + 1)
    else:
        return(i - 1, i + 1)

In [38]:
V = np.copy(K)

In [47]:
E = 0
T = 1
d_max = D.max()
for X in range(cd_num):
    Uxi_exp_accum = 0
    Uxi_ls = []
    for i in range(aa_num):
        cstr = 0
        dist = 0
        # calculate Uxi
        for Y in range(cd_num):
            if Y != X:
                cstr += V[Y, i]
                dist += D[X, Y] * (V[Y, I(i)[0]] + V[Y, I(i)[1]])
        Uxi = - d_max * cstr - dist
        
        Uxi_ls.append(Uxi)
        Uxi_exp_accum += np.exp(Uxi / T)
    # update Vxi
    for i in range(aa_num):
        V[X, i] = np.exp(Uxi_ls[i] / T) / Uxi_exp_accum
        
        # calculate E
        E += - V[X, i] * Uxi_ls[i]
E

8093.3852915747575

In [171]:
pd.DataFrame(np.round(V), columns=aa, index=cd).astype(int)

,A,E,Q,L,F,Y,R,G,K,D,...,V,C,X,P,S,M,I,W,N,T
AAA,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAC,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAG,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAT,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
ACC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
ACG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
ACT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
AGA,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
AGC,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [150]:
def annealing_runner(V_matrix, D_max, temp, E_thrhd):
    T = temp # declare temperature T
    d_max = D_max # declare the weight of colunm constraint
    n = 0 # a counter to count epochs
    delta_E = 100 # define delta_E
    e = 0 # declare energy e in the previous round
    V = np.random.random([len(cd), len(aa)])
    # start simulate loop, the loop stops while the delta energy is less than the specified threshold
    while delta_E > E_thrhd:
        E = 0 # declare energy E
        
        # count epochs
        n += 1
        
        # interrupt the loop if E becomes a nan value
        if np.isnan(E):
            E = 0
            break
        for X in range(cd_num):
            Uxi_exp_accum = 0
            Uxi_ls = []
            for i in range(aa_num):
                cstr = 0
                dist = 0
                # calculate Uxi
                for Y in range(cd_num):
                    if Y != X:
                        cstr += V[Y, i]
                        dist += D[X, Y] * (V[Y, I(i)[0]] + V[Y, I(i)[1]])
                Uxi = - d_max * cstr - dist
        
                Uxi_ls.append(Uxi)
                Uxi_exp_accum += np.exp(Uxi / T)
            # update Vxi
            for i in range(aa_num):
                V[X, i] = np.exp(Uxi_ls[i] / T) / Uxi_exp_accum
                # calculate energy E
                E += - V[X, i] * Uxi_ls[i]
                
        # update delta_E
        delta_E = np.absolute(E - e)
        # update previous energy e
        e = E
        
    return(E, n)
annealing_runner(V, 32, 3.6, 0.01)

(7952.2818499350669, 26)

In [106]:
V

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       ..., 
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])

In [132]:
V = np.random.random([len(cd), len(aa)])
J = annealing_runner(V, 32, 1.2, 0.01)

C:\Users\chong\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


In [134]:
list(J)

[nan, 1]

In [131]:
len(np.unique(np.where(V.round() == 1)[0]))

64

In [137]:
np.isnan(list(V)).any()

True

In [60]:
np.isnan(J[0, 0])

True

In [126]:
pd.DataFrame(np.round(V), columns=aa, index=cd).astype(int)

,Y,P,D,G,N,R,E,Q,M,K,...,S,A,W,L,I,C,X,V,F,H
AAA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
AAC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
AAG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
AAT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
ACA,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACC,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACG,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ACT,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AGA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
AGC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [29]:
def annealing_runner(D_max, temp, E_thrhd):
    T = temp # declare temperature T
    d_max = D_max # declare the weight of colunm constraint
    n = 0 # a counter to count epochs
    delta_E = 100 # define delta_E
    e = 0 # declare energy e in the previous round
    V = np.random.random([len(cd), len(aa)])
    
    # start simulate loop, the loop stops while the delta energy is less than the specified threshold
    while delta_E > E_thrhd:
        E = 0 # declare energy E
        
        # count epochs
        n += 1
        
        # interrupt the loop if E becomes a nan value
        if np.isnan(E):
            E = 0
            break
        for X in range(cd_num):
            Uxi_exp_accum = 0
            Uxi_ls = []
            for i in range(aa_num):
                cstr = 0
                dist = 0
                # calculate Uxi
                for Y in range(cd_num):
                    if Y != X:
                        cstr += V[Y, i]
                        dist += D[X, Y] * (V[Y, I(i)[0]] + V[Y, I(i)[1]])
                Uxi = - d_max * cstr - dist
        
                Uxi_ls.append(Uxi)
                Uxi_exp_accum += np.exp(Uxi / T)
            # update Vxi
            for i in range(aa_num):
                V[X, i] = np.exp(Uxi_ls[i] / T) / Uxi_exp_accum
                # calculate energy E
                E += - V[X, i] * Uxi_ls[i]
                
        # update delta_E
        delta_E = np.absolute(E - e)
        # update previous energy e
        e = E
    return(E, n, V)

def simulator(D_range, T_range, accuracy, iteration):
    out_dict = {}
    # iterate over every possible pair of dmax & T
    for dmax in np.arange(D_range[0], D_range[1], D_range[2]):
        for temp in np.arange(T_range[0], T_range[1], T_range[2]):
            out_dict[(dmax, temp)] = {}
            out_dict[(dmax, temp)]['record'] = {}
            
            # initialize a variable to compute rate of valid outcomes
            valid_num = 0
            E_ls = []
            n_ls = []
            for i in range(iteration):
                out_dict[(dmax, temp)]['record'][i] = {}
                annealing_out = list(annealing_runner(dmax * D.max(), temp, accuracy))
                n_ls.append(annealing_out[1])
                
                # examine the validity of outputs
                if np.isnan(annealing_out[0]).any(): # an output is invalid if it has 'nan' values
                    out_dict[(dmax, temp)]['record'][i]['E'] = -1
                    out_dict[(dmax, temp)]['record'][i]['V'] = np.NaN
                else:
                    if len(np.unique(np.where(annealing_out[2].round() == 1)[0])) != 64: # an output is invalid if it doesn't satisfy row constraints
                        out_dict[(dmax, temp)]['record'][i]['E'] = 0
                        out_dict[(dmax, temp)]['record'][i]['V'] = np.NaN
                    else:
                        out_dict[(dmax, temp)]['record'][i]['E'] = annealing_out[0]
                        out_dict[(dmax, temp)]['record'][i]['V'] = (annealing_out[2].round()).astype(int)
                        E_ls.append(annealing_out[0])
            print("D_max=%f\tTemperature=%f" % (dmax, temp))
    return(out_dict)
J = simulator([1.8, 0.8, -0.2], [2.6, 4.2, 0.2], 0.01, 1)

C:\Users\chong\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars


D_max=1.800000	Temperature=2.600000
D_max=1.800000	Temperature=2.800000
D_max=1.800000	Temperature=3.000000
D_max=1.800000	Temperature=3.200000
D_max=1.800000	Temperature=3.400000
D_max=1.800000	Temperature=3.600000
D_max=1.800000	Temperature=3.800000
D_max=1.800000	Temperature=4.000000
D_max=1.600000	Temperature=2.600000
D_max=1.600000	Temperature=2.800000
D_max=1.600000	Temperature=3.000000
D_max=1.600000	Temperature=3.200000
D_max=1.600000	Temperature=3.400000
D_max=1.600000	Temperature=3.600000
D_max=1.600000	Temperature=3.800000
D_max=1.600000	Temperature=4.000000
D_max=1.400000	Temperature=2.600000
D_max=1.400000	Temperature=2.800000
D_max=1.400000	Temperature=3.000000
D_max=1.400000	Temperature=3.200000
D_max=1.400000	Temperature=3.400000
D_max=1.400000	Temperature=3.600000
D_max=1.400000	Temperature=3.800000
D_max=1.400000	Temperature=4.000000
D_max=1.200000	Temperature=2.600000
D_max=1.200000	Temperature=2.800000
D_max=1.200000	Temperature=3.000000
D_max=1.200000	Temperature=3

In [26]:
J

{(1.0000000000000002,
  2.6000000000000001): {'record': {0: {'E': -1, 'V': nan}}},
 (1.0000000000000002,
  2.8000000000000003): {'record': {0: {'E': -1, 'V': nan}}},
 (1.0000000000000002,
  3.0000000000000004): {'record': {0: {'E': 7286.2948458202627,
    'V': array([[0, 1, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           [0, 0, 1, ..., 0, 0, 0],
           ..., 
           [0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 1, 0],
           [0, 0, 0, ..., 1, 0, 0]])}}},
 (1.0000000000000002,
  3.2000000000000006): {'record': {0: {'E': 7792.7735969089317,
    'V': array([[0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           ..., 
           [0, 0, 1, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0]])}}},
 (1.0000000000000002,
  3.4000000000000008): {'record': {0: {'E': 7627.4460375680628,
    'V': array([[0, 0, 0, ..., 1, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           [0

In [13]:
list(J.keys())[0][0] == 1.6

True

In [27]:
df = pd.DataFrame(0, index=np.arange(1.8, 0.8, -0.2), columns=np.arange(2.6, 4.2, 0.2))
df

,2.6,2.8,3.0,3.2,3.4,3.6,3.8,4.0
1.8,0,0,0,0,0,0,0,0
1.6,0,0,0,0,0,0,0,0
1.4,0,0,0,0,0,0,0,0
1.2,0,0,0,0,0,0,0,0
1.0,0,0,0,0,0,0,0,0


In [30]:
for dmax in np.arange(1.8, 0.8, -0.2):
    for temp in np.arange(2.6, 4.2, 0.2):
        df.loc[dmax, temp] = J[(dmax, temp)]['record'][0]['E']
df

,2.6,2.8,3.0,3.2,3.4,3.6,3.8,4.0
1.8,-1,-1,-1.0,-1.000000,-1.000000,-1.000000,11400.181436,-1.000000
1.6,-1,-1,-1.0,-1.000000,-1.000000,-1.000000,10420.700507,10588.687568
1.4,-1,-1,-1.0,-1.000000,9367.502869,9358.103526,9201.508368,9554.770067
1.2,-1,-1,-1.0,0.000000,9051.379595,8916.659297,8582.295038,9088.685461
1.0,-1,-1,-1.0,7491.358233,7494.822980,7321.817487,0.000000,7632.626571


In [25]:
#!usr/local/bin python
import sys
import numpy as np
import pandas as pd
from Bio.Data import CodonTable

# specify parameter for simulator
# *************************************
Dmax_range = [1.8, 1.2, -0.2]
Temp_range = [3.2, 4.0, 0.2]
accuracy = 0.01
replicates = 2
# *************************************

# progress_calculator
def total_progress(dmax_range, temp_range, replicates):
    d_num = np.absolute((dmax_range[0] - dmax_range[1]) / dmax_range[2])
    t_num = np.absolute((temp_range[0] - temp_range[1]) / temp_range[2])
    total_num = d_num * t_num * replicates
    return(total_num)

# initialize codon table & global variables
standard_table = CodonTable.unambiguous_dna_by_name["Standard"]
codon_table = standard_table.forward_table
for codon in standard_table.stop_codons:
    codon_table[codon] = 'X'

cd = sorted(list(codon_table.keys()))
aa = list(set(codon_table.values()))
cd_num = len(cd)
aa_num = len(aa)

# distance calculator
def distance(A1, A2):
    D = 0
    for k in range(3):
        # transition
        if set([A1[k], A2[k]]) in [set(['A', 'G']), set(['C', 'T'])]:
            if k == 0 and A1[k] != A2[k]:
                D += 5
            elif k == 1 and A1[k] != A2[k]:
                D += 10
            elif k == 2 and A1[k] != A2[k]:
                D += 1
        # transversion
        else:
            if k == 0 and A1[k] != A2[k]:
                D += 10
            elif k == 1 and A1[k] != A2[k]:
                D += 20
            elif k == 2 and A1[k] != A2[k]:
                D += 2
    return(D)

# compute distance matrix
def D_matrix():
    D = np.zeros([cd_num, cd_num])
    for i in range(cd_num):
        for j in range(cd_num):
            D[i, j] = distance(cd[i], cd[j])
    D = D.astype(int)
    return(D)
D = D_matrix()

# compute subtitle for V matrix
def I(i):
    if i + 1 > aa_num - 1:
        return(i - 1, 0)
    elif i - 1 < 0:
        return(aa_num - 1, i + 1)
    else:
        return(i - 1, i + 1)
    
# initialize annealing model
def annealing_runner(D_max, temp, E_thrhd):
    T = temp # declare temperature T
    d_max = D_max # declare the weight of colunm constraint
    delta_E = 100 # define delta_E
    e = 0 # declare energy e in the previous round
    V = np.random.random([len(cd), len(aa)])

    # start simulate loop, the loop stops while the delta energy is less than the specified threshold
    while delta_E > E_thrhd:
        E = 0 # declare energy E
        # interrupt the loop if E becomes a nan value
        if np.isnan(E):
            E = 0
            break
        for X in range(cd_num):
            Uxi_exp_accum = 0
            Uxi_ls = []
            for i in range(aa_num):
                cstr = 0
                dist = 0
                # calculate Uxi
                for Y in range(cd_num):
                    if Y != X:
                        cstr += V[Y, i]
                        dist += D[X, Y] * (V[Y, I(i)[0]] + V[Y, I(i)[1]])
                Uxi = - d_max * cstr - dist

                Uxi_ls.append(Uxi)
                Uxi_exp_accum += np.exp(Uxi / T)
            # update Vxi
            for i in range(aa_num):
                V[X, i] = np.exp(Uxi_ls[i] / T) / Uxi_exp_accum
                # calculate energy E
                E += - V[X, i] * Uxi_ls[i]

        # update delta_E
        delta_E = np.absolute(E - e)
        # update previous energy e
        e = E
    return(E, V)

# initialize simulator
def simulator(D_range, T_range, accuracy, iteration):
    progress = 0
    out_dict = {}
    t_progress = total_progress(D_range, T_range, iteration)
    # iterate over every possible pair of dmax & T
    for dmax in np.arange(D_range[0], D_range[1], D_range[2]):
        for temp in np.arange(T_range[0], T_range[1], T_range[2]):
            valid_num = 0
            out_dict[(dmax, temp)] = {}
            out_dict[(dmax, temp)]['record'] = {}

            for i in range(iteration):
                out_dict[(dmax, temp)]['record'][i] = {}
                annealing_out = list(annealing_runner(dmax * D.max(), temp, accuracy))

                # examine the validity of outputs
                if np.isnan(annealing_out[0]).any(): # an output is invalid if it has 'nan' values
                    out_dict[(dmax, temp)]['record'][i]['E'] = -1
                    out_dict[(dmax, temp)]['record'][i]['V'] = np.NaN
                else:
                    if len(np.unique(np.where(annealing_out[1].round() == 1)[0])) != 64: # an output is invalid if it doesn't satisfy row constraints
                        out_dict[(dmax, temp)]['record'][i]['E'] = 0
                        out_dict[(dmax, temp)]['record'][i]['V'] = np.NaN
                    else:
                        out_dict[(dmax, temp)]['record'][i]['E'] = annealing_out[0]
                        out_dict[(dmax, temp)]['record'][i]['V'] = (annealing_out[1].round()).astype(int)
                        valid_num += 1
                print("D_max: %.1f\tTemperature: %.1f\tReplicate: V.%d, complete!" % (dmax, temp, i), end='\r', flush=True)
            out_dict[(dmax, temp)]['valid_rate'] = valid_num / iteration * 100
    print("\nDone!")
    return(out_dict)
# dump outputs
outputs = simulator(Dmax_range, Temp_range, accuracy, replicates)

C:\Users\chong\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:106: RuntimeWarning: invalid value encountered in double_scalars


D_max: 1.2	Temperature: 3.8	Replicate: 1 complete!
Done!


In [26]:
def results_writer(out_dict):
    df = pd.DataFrame(0, index=np.arange(Dmax_range[0], Dmax_range[1], Dmax_range[2]), columns=np.arange(Temp_range[0], Temp_range[1], Temp_range[2]))
    for dt_pair in out_dict.keys():
        energy = 0
        n = 0
        for ix in out_dict[dt_pair]['record'].keys():
            E = out_dict[dt_pair]['record'][ix]['E']
            if E != -1 and E != 0:
                energy += E
                n += 1
        if n != 0:
            df.loc[dt_pair[0], dt_pair[1]] = str(np.round(energy / n, 2)) + " ({0}%%)".format(out_dict[dt_pair]['valid_rate'])
        else:
            df.loc[dt_pair[0], dt_pair[1]] = 0
    return(df)
results_writer(outputs)

,3.2,3.4,3.6,3.8
1.8,0,0,0,0
1.6,0,0,10612.92 (100.0%%),10365.05 (50.0%%)
1.4,0,0,9978.04 (100.0%%),9696.35 (50.0%%)
1.2,8718.05 (100.0%%),8547.14 (100.0%%),8667.92 (100.0%%),9158.83 (100.0%%)


In [13]:
outputs

{(1.2000000000000002,
  3.2000000000000002): {'record': {0: {'E': 9065.3003188457187,
    'V': array([[0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           ..., 
           [0, 0, 0, ..., 0, 1, 0],
           [0, 0, 0, ..., 0, 0, 1],
           [1, 0, 0, ..., 0, 0, 0]])},
   1: {'E': 8696.4156628610817, 'V': array([[0, 0, 0, ..., 0, 0, 1],
           [0, 0, 0, ..., 0, 0, 1],
           [0, 0, 0, ..., 1, 0, 0],
           ..., 
           [0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0]])}}, 'valid_rate': 1.0},
 (1.2000000000000002,
  3.4000000000000004): {'record': {0: {'E': 0, 'V': nan},
   1: {'E': 8524.4146662999701, 'V': array([[0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           [0, 0, 0, ..., 0, 0, 0],
           ..., 
           [0, 1, 0, ..., 0, 0, 0],
           [1, 0, 0, ..., 0, 0, 0],
           [1, 0, 0, ..., 0, 0, 0]])}}, 'valid_rate': 0.5},
 (1.200000000

In [29]:
np.where(outputs[(1.2000000000000002,3.2000000000000002)]['record'][0]['V'] == 1)[0]

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63], dtype=int64),
 array([ 9,  9, 10, 10, 14, 15, 15, 16, 19, 19, 20, 18,  7,  7,  8,  8,  3,
         2,  1,  3, 13, 12, 14, 13, 20,  0,  0, 20,  4,  5,  4,  5, 11, 10,
        10, 11, 17, 16, 15, 16, 18, 18, 19, 17,  7,  7,  6,  8,  2,  2,  3,
         2, 13, 12, 14, 13,  0,  1,  0,  1,  5,  6,  5,  6], dtype=int64))

In [31]:
Dmax_range

AttributeError: 'list' object has no attribute 'round'